<a href="https://colab.research.google.com/github/siribafna/Myocardial-Infarction-Predictions/blob/main/variables_113_125_cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
import pandas as pd 
MI_data = pd.read_csv('https://raw.githubusercontent.com/siribafna/Myocardial-Infarction-Predictions/main/MI.txt')

In [ ]:
MI_data.describe()

In [ ]:
# count the number of missing values for each column
total_missing = MI_data.isnull().sum()
missing = total_missing > (len(MI_data)*.5)
missing = total_missing[missing]

# get variables with NaN values > 50% of MI_data
print(missing)
missing[1]
# report the results
total_missing.to_csv('out.csv', index=True)  

In [ ]:
MI_data.drop(missing.index, axis=1)

In [77]:
# fill missing values with mean column values
MI_data.fillna(MI_data.mean(), inplace=True)

In [113]:
import numpy as np
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import seaborn as sns
import io 

data1 = pd.read_csv('https://raw.githubusercontent.com/siribafna/Myocardial-Infarction-Predictions/main/MI-cleaned-TT.csv', sep = ',')

data1.head()

,ID,AGE,SEX,INF_ANAM,STENOK_AN,FK_STENOK,CHD,GB,SIM_GIPERT,DLIT_AG,...,JELUD_TAH,FIBR_JELUD,A_V_BLOK,OTEK_LANC,RAZRIV,DRESSLER,ZSN,REC_IM,P_IM_STEN,LET_IS
0,1,77.0,1,2.0,1.0,1.0,2.0,3.0,0.0,7.0,...,0,0,0,0,0,0,0,0,0,0
1,2,55.0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,3,52.0,1,0.0,0.0,0.0,2.0,2.0,0.0,2.0,...,0,0,0,0,0,0,0,0,0,0
3,4,68.0,0,0.0,0.0,0.0,2.0,2.0,0.0,3.0,...,0,0,0,0,0,0,1,0,0,0
4,5,60.0,1,0.0,0.0,0.0,2.0,3.0,0.0,7.0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Heatmap

plt.figure(figsize = (100, 100))
cor = data1.corr()
sns.heatmap(cor, annot = True, cmap = plt.cm.Reds)
plt.show()

In [ ]:
# Correlation with output variable
cor_target = abs(cor["LET_IS"])

# Selecting highly correlated features
relevant_features = cor_target[cor_target > 0.5]
relevant_features

In [96]:
# Original % for complications
data1.REC_IM.value_counts() / len(data1)


0    0.907825
1    0.092175
Name: REC_IM, dtype: float64

In [114]:
# Create new dataframe for complications
newset = data1[['FIBR_PREDS', 'PREDS_TAH', 'JELUD_TAH', 'FIBR_JELUD', 'A_V_BLOK',
                'OTEK_LANC', 'RAZRIV', 'DRESSLER', 'ZSN', 'REC_IM', 'P_IM_STEN']].copy()
newset.index.name = 'count'
newset.rename(columns = {'PREDS_TAH':'complications'}, inplace = True)

# Add columns
newset.complications = newset.complications.astype(int) + newset.pop('FIBR_PREDS').astype(int) + newset.pop('OTEK_LANC').astype(int) 
+ newset.pop('P_IM_STEN').astype(int) + newset.pop('A_V_BLOK').astype(int) + newset.pop('JELUD_TAH').astype(int) + newset.pop('REC_IM').astype(int) 
+ newset.pop('ZSN').astype(int) + newset.pop('DRESSLER').astype(int) + newset.pop('FIBR_JELUD').astype(int) + newset.pop('RAZRIV').astype(int)

#newset["FIBR_PREDS"].where(~pd.isnull(newset["FIBR_PREDS"]), newset["PREDS_TAH"])
#newset[["FIBR_PREDS", "PREDS_TAH"]].max(axis=1)

# Convert binary (1/0) to yes/no
newset = newset.replace({1 : 'yes', 0 : 'no', 2 : 'yes', 3 : 'yes'})

newset1 = newset[['complications']].copy()
newset1
             

,complications
count,
0,no
1,no
2,no
3,no
4,no
...,...
1503,no
1504,yes
1505,no


In [95]:
# Check new complications %
newset1.complications.value_counts() / len(newset)


no     0.812334
yes    0.187666
Name: complications, dtype: float64

In [123]:
# Add complications column
addcolumn = newset1['complications']
data1 = pd.concat([data1, addcolumn], axis = 1)

# Remove previous columns
dropped = data1.drop(columns = ['FIBR_PREDS', 'PREDS_TAH', 'JELUD_TAH', 'FIBR_JELUD', 'A_V_BLOK',
             'OTEK_LANC', 'RAZRIV', 'DRESSLER', 'ZSN', 'REC_IM', 'P_IM_STEN'])



In [124]:
# Before combining
dropped.LET_IS.value_counts() / len(dropped)

0    0.861406
1    0.046419
3    0.032493
7    0.015915
4    0.013926
6    0.013263
2    0.009284
5    0.007294
Name: LET_IS, dtype: float64

In [121]:
LET_IS = dropped.replace({0 : 'alive', 1 : 'death', 2 : 'death', 3 : 'death', 4 : 'death', 5 : 'death', 6 : 'death', 7 : 'death'})

dropped

# % After combining
dropped.LET_IS.value_counts() / len(dropped)

alive    0.861406
death    0.138594
Name: LET_IS, dtype: float64